<a href="https://colab.research.google.com/github/Sirabhop/CLM/blob/master/CLM_Administration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

In [16]:
import pandas as pd
import os
import gspread

In [5]:
gc = gspread.service_account('/content/drive/My Drive/My CLM project/Machine Learning/credential/infinite-deck-288214-a013e2afcf8a.json')
sheet = gc.open('CLM Note')

Admin = sheet.worksheet('Admin')
Admin = pd.DataFrame(Admin.get_all_records())

In [23]:
os.chdir("/content/drive/My Drive/My CLM project/Machine Learning")

In [38]:
usecols = range(1,27)
CallReport = pd.read_excel("Call Report CLM Date 01-30 Aug 2020.xlsx", header = 2,
                           usecols = usecols)

In [51]:
CallReport.iloc[0, 6:17].values

array(['สมัครบริการ Offer 1', '% Success Offer 1', 'สมัครบริการ Offer 2',
       '% Success Offer 2', 'Total สมัครบริการ', 'Call Back', 'Bad Lead',
       'Follow Up', 'สนใจแต่ติดเงื่อนไข', 'ปฏิเสธ', 'After Sale'],
      dtype=object)

In [54]:
CallReport.columns

Index(['Campaign No.', 'Campaign Name', 'Owner', 'LIFESTAGE', 'LIFESTYLE',
       'Total Lead', 'ติดต่อได้', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', '%Success',
       'Answer', 'No Answer', 'Grand Total', '% Answer', '% No Answer', 'Prod',
       'Prod type'],
      dtype='object')

In [58]:
CallReport.rename(columns = {'ติดต่อได้':'สมัครบริการ Offer 1', 'Unnamed: 8':'% Success Offer 1', 'Unnamed: 9':'สมัครบริการ Offer 2', 
                             'Unnamed: 10':'% Success Offer 2', 'Unnamed: 11':'Total สมัครบริการ', 'Unnamed: 12':'Call Back', 
                             'Unnamed: 13':'Bad Lead', 'Unnamed: 14':'Follow Up', 'Unnamed: 15':'สนใจแต่ติดเงื่อนไข', 'Unnamed: 16':'ปฏิเสธ', 
                             'Unnamed: 17':'After Sale', 'Unnamed: 18':'สายหลุดระหว่างสนทนา'}, inplace = True).drop(index = 0, axis = 0, inplace = True)